In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, Distance, VectorParams, ScoredPoint, Filter, FieldCondition, MatchValue
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import hashlib
from datetime import datetime, timezone
import os
import uuid
import numpy as np
from bson import ObjectId
import torch
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import spearmanr

/home/sagar/pipeline OS/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Connections

In [2]:
from pymongo import MongoClient

# mongo conection

mongo_uri = "mongodb://localhost:27017/" #compass
mongo_client = MongoClient(mongo_uri)
db = mongo_client['pipelineos']

# loading the mongodb into variables:

icp_profiles = db["icp_profiles"]
discovered_companies = db["discovered_companies"]
scored_companies = db['scored_companies']

#### Qdrant connections

In [3]:
# qdrant connection via https

from qdrant_client.models import VectorParams, Distance
qdrant = QdrantClient(
    host="localhost",
    port=6333)
#using qdrants https method as used 6333 port and gRPC is :
# qdrant = QdrantClient(
#     host="localhost", 
#     port=6334, 
#     grpc=True
# )

#### Run the Qdrant

In [4]:
# quadrant setup via docker:
# run this in ubuntu terminal:

# docker pull qdrant/qdrant
# docker run -p 6333:6333 -p 6334:6334 qdrant/qdrant

#### Models

In [5]:
# # vectorize setup:

# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') #384-dim vector
# model = SentenceTransformer("BAAI/bge-small-en")  # 384 D
model = SentenceTransformer("all-MiniLM-L12-v2") # 384 D
# model = SentenceTransformer("all-mpnet-base-v2") # 768 d
# model = SentenceTransformer("all-distilroberta-v1") # 768
# model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1") # 384 d

#### To fetch the latest ICP from the MongoDB

In [6]:
# Aggregation pipeline to get the latest active ICP
pipeline = [
    {"$match": {"active": True}},
    {"$addFields": {
        "effective_time": {
            "$cond": {
                "if": {"$gt": ["$updated_at", "$created_at"]},
                "then": "$updated_at",
                "else": "$created_at"
            }
        }
    }},
    {"$sort": {"effective_time": -1}},
    {"$limit": 1}
]

latest_icp = list(icp_profiles.aggregate(pipeline))
if latest_icp:
    latest_icp = latest_icp[0]
    print("Latest active ICP:")
    print(latest_icp)
else:
    print("No active ICP found.")

Latest active ICP:
{'_id': ObjectId('6899cb3aafa62213309e0357'), 'name': 'google', 'filters': {'locations': ['usa', 'canada'], 'tech_stack': ['AWS', 'python'], 'keywords': ['ai, ml'], 'designations': ['hr'], 'department': ['tect', 'aq', 'it'], 'industry': ['software', 'ai'], 'employee_count': {'min': 1, 'max': 500}, 'founded_after': 2019}, 'tags': ['tech', 'software'], 'active': True, 'created_at': datetime.datetime(2025, 8, 11, 10, 51, 38, 658000), 'updated_at': datetime.datetime(2025, 8, 11, 10, 51, 38, 658000), 'effective_time': datetime.datetime(2025, 8, 11, 10, 51, 38, 658000)}


In [7]:
latest_icp

{'_id': ObjectId('6899cb3aafa62213309e0357'),
 'name': 'google',
 'filters': {'locations': ['usa', 'canada'],
  'tech_stack': ['AWS', 'python'],
  'keywords': ['ai, ml'],
  'designations': ['hr'],
  'department': ['tect', 'aq', 'it'],
  'industry': ['software', 'ai'],
  'employee_count': {'min': 1, 'max': 500},
  'founded_after': 2019},
 'tags': ['tech', 'software'],
 'active': True,
 'created_at': datetime.datetime(2025, 8, 11, 10, 51, 38, 658000),
 'updated_at': datetime.datetime(2025, 8, 11, 10, 51, 38, 658000),
 'effective_time': datetime.datetime(2025, 8, 11, 10, 51, 38, 658000)}

In [8]:
for i in discovered_companies.find({}):
    print(i)

{'_id': ObjectId('68890042ada3746b1bacf2a5'), 'name': 'Flowcast AI solutions', 'domain': 'flowcast.ai', 'industries': ['Artificial Intelligence', 'FinTech'], 'employee_count_estimate': {'min': 50, 'max': 200}, 'hq_location': {'city': 'San Francisco', 'country': 'USA'}, 'founded_year': 2019, 'tech_stack': ['Python', 'AWS', 'LangChain'], 'social_metrics': {'followers': {'linkedin': 1500, 'twitter': 600}}, 'source_urls': ['https://linkedin.com/company/flowcast-ai'], 'last_scraped': datetime.datetime(2025, 7, 27, 11, 0), 'last_scored': None, 'last_vectorized': datetime.datetime(2025, 8, 4, 10, 32, 19, 574000), 'changed_at': datetime.datetime(2025, 8, 3, 10, 0), 'icp_id': ObjectId('64c4a23b69a84f2b0b789cde'), 'icp_version': 1}
{'_id': ObjectId('68890042ada3746b1bacf2a6'), 'name': 'SynthAI Inc.', 'domain': 'ssynthai.io', 'industries': ['Artificial Intelligence', 'SaaS'], 'employee_count_estimate': {'min': 20, 'max': 80}, 'hq_location': {'city': 'Toronto', 'country': 'Canada'}, 'founded_year'

#### Delta Ware Filter

In [9]:
latest_icp = list(icp_profiles.aggregate(pipeline))
companies_to_score = []

for icp in latest_icp:
    icp_id = str(icp["_id"])
    icp_version = icp.get("version", 1)

    for company in discovered_companies.find({}):
        company_id = company["_id"]
        last_scraped = company["last_scraped"]


        # Look up the scored company metadata from separate collection
        score_doc = scored_companies.find_one({
            "company_id": company_id,
            "icp_id": icp_id
        })

        # CASE A: Never scored for this ICP
        if not score_doc:
            companies_to_score.append({"company": company, "icp": icp})
            continue

        # CASE B: Rescraped after last scoring
        last_scored = score_doc.get("last_scored")
        if last_scored is None or last_scored > last_scraped: # logic changed just to check, reverse it once the task is done
            companies_to_score.append({"company": company, "icp": icp})
            continue

        # CASE C: ICP version has changed
        scored_version = score_doc.get("icp_version")
        if scored_version != icp_version:
            companies_to_score.append({"company": company, "icp": icp})
            continue

print(f"Total <company, ICP> pairs to score: {len(companies_to_score)}")


Total <company, ICP> pairs to score: 5


### Collections Creation in Qdrant

In [10]:
# === First: Create collection if it doesn't exist ===


# if qdrant.collection_exists("icp_vectors"):
#     qdrant.delete_collection("icp_vectors")

# ICP_Vectors

if not qdrant.collection_exists("icp_vectors"):
    qdrant.create_collection(
        collection_name="icp_vectors",
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)  # 384 for MiniLM
    )



# if qdrant.collection_exists("company_vectors"):
#     qdrant.delete_collection("company_vectors")

# Company_Vectors

if not qdrant.collection_exists("company_vectors"):
    qdrant.create_collection(
        collection_name="company_vectors",
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)
    )

#### Company Vectorization and upsert with filter 

In [11]:
# import datetime
# import numpy as np
# import uuid
# from pymongo import UpdateOne
# from qdrant_client.models import PointStruct
# from bson import ObjectId

# def normalize_list(value):
#     if isinstance(value, str):
#         return [value.lower()]
#     if isinstance(value, list):
#         return list({v.strip().lower() for v in value if isinstance(v, str)})
#     return []

# def prepare_vector_text(company):
#     parts = []
#     parts += normalize_list(company.get("industries"))
#     parts += normalize_list(company.get("tech_stack"))

#     country = company.get("hq_location", {}).get("country", "").lower()
#     if country:
#         parts.append(country)

#     return " ".join(parts).strip()

# points = []
# bulk_updates = []

# for pair in companies_to_score:
#     company = pair["company"]
#     icp = pair["icp"]

#     domain = company.get("domain")
#     if not domain:
#         continue

#     # changed_at = company.get("changed_at")
#     # last_vectorized = company.get("last_vectorized")

#     # if changed_at and last_vectorized:
#     #     if changed_at < last_vectorized:
#     #         print(f"Skipping {domain}: No changes since last vectorization.")
#     #         print(f"[DEBUG] {domain} | changed_at: {changed_at}, last_vectorized: {last_vectorized}")

#     #         continue

#     vector_text = prepare_vector_text(company)
#     if not vector_text:
#         print(f"Skipping {domain}: No vector text.")
#         continue

#     try:
#         embedding = model.encode(vector_text)
#         if embedding is None or not isinstance(embedding, (list, np.ndarray)) or np.all(np.array(embedding) == 0):
#             print(f"Skipping {domain}: Invalid embedding.")
#             continue

#         embedding = np.array(embedding, dtype=np.float32).tolist()
#         point_id = str(uuid.uuid5(uuid.NAMESPACE_DNS, f"company:{domain}"))

#         point = PointStruct(
#             id=point_id,
#             vector=embedding,
#             payload={
#                 "company_id": str(company.get("_id")),
#                 "icp_id": str(company.get("icp_id", "")),
#                 "industry": normalize_list(company.get("industries")),
#                 "location": company.get("hq_location", {}).get("country", "").lower(),
#                 "tech_stack": normalize_list(company.get("tech_stack"))
#             }
#         )
#         points.append(point)

#         bulk_updates.append(UpdateOne(
#             {"_id": company["_id"]},
#             {"$set": {"last_vectorized": datetime.datetime.now(datetime.timezone.utc)}}
#         ))

#     except Exception as e:
#         print(f"Error vectorizing {domain}: {e}")

# if points:
#     qdrant.upsert(collection_name="company_vectors", points=points)
#     print(f"Upserted {len(points)} companies to Qdrant.")
# else:
#     print("No valid company vectors to upsert.")

# if bulk_updates:
#     db["discovered_companies"].bulk_write(bulk_updates)
#     print(f"Updated last_vectorized for {len(bulk_updates)} companies.")


#### ICP Vectorization and upsert with filter 

In [12]:
# from datetime import datetime
# from uuid import uuid5, NAMESPACE_DNS
# import numpy as np
# from qdrant_client.models import PointStruct

# # === Helper: Normalize and join values ===
# def normalize_list(value):
#     if isinstance(value, str):
#         return [value.lower()]
#     if isinstance(value, list):
#         return list({v.strip().lower() for v in value if isinstance(v, str)})
#     return []

# # === Load active ICPs ===
# latest_icp = list(icp_profiles.aggregate(pipeline))
# icp_points = []

# for icp in latest_icp:
#     icp_id = str(icp["_id"])
#     filters = icp.get("filters", {})
#     updated_at = icp.get("updated_at", datetime.min)
#     last_vectorized = icp.get("last_vectorized")  # May be None

#     # === Skip if already vectorized and not updated ===
#     # if last_vectorized:
#     #     if updated_at < last_vectorized:
#     #         print(f"Skipping ICP {icp_id}: No changes since last vectorization.")
#     #         continue

#     # === Normalize and build vector text ===
#     text_parts = []

#     industries = normalize_list(filters.get("industries", []))
#     locations = normalize_list(filters.get("locations", []))
#     tech_stack = normalize_list(icp.get("tech_stack", []))
#     keywords = normalize_list(icp.get("keywords", []))

#     text_parts += industries + locations + tech_stack + keywords
#     vector_text = " ".join(text_parts).strip()

#     if not vector_text:
#         continue

#     try:
#         embedding = model.encode(vector_text)
#         if embedding is None or not isinstance(embedding, (list, np.ndarray)) or np.all(np.array(embedding) == 0):
#             continue

#         embedding = np.array(embedding, dtype=np.float32).tolist()
#         point_id = str(uuid5(NAMESPACE_DNS, f"icp:{icp_id}"))

#         point = PointStruct(
#             id=point_id,
#             vector=embedding,
#             payload={
#                 "icp_id": icp_id,
#                 "industry": industries,
#                 "location": locations,
#                 "tech_stack": tech_stack,
#                 "keywords": keywords
#             }
#         )
#         icp_points.append(point)

#         # === Update last_vectorized timestamp in MongoDB ===
#         # icp_profiles.update_one(
#         #     {"_id": icp["_id"]},
#         #     {"$set": {"last_vectorized": datetime.utcnow()}}
#         # )

#     except Exception as e:
#         print(f"Error encoding ICP {icp_id}: {e}")

# # === Final upsert to Qdrant ===
# if icp_points:
#     qdrant.upsert(collection_name="icp_vectors", points=icp_points)
#     print(f"Upserted {len(icp_points)} ICP profiles to Qdrant.")
# else:
#     print("No valid ICP vectors to upsert.")


#### without filter ICP and Company Vectors:

In [13]:
# === Helper: Normalize and join string lists ===
def normalize_list(value):
    if isinstance(value, str):
        return [value.lower()]
    if isinstance(value, list):
        return list({v.strip().lower() for v in value if isinstance(v, str)})
    return []

# === Helper: Prepare vector text from normalized fields ===
def prepare_vector_text(company):
    parts = []

    industries = normalize_list(company.get("industries"))
    tech_stack = normalize_list(company.get("tech_stack"))
    country = company.get("hq_location", {}).get("country", "").lower()

    parts += industries
    parts += tech_stack
    if country:
        parts.append(country)

    return " ".join(parts).strip()

# === Vectorize and Upsert to Qdrant ===
points = []
for pair in companies_to_score:
    company = pair["company"]
    icp = pair["icp"]

    domain = company.get("domain")
    if not domain:
        continue

    # === Check vector freshness ===
    # changed_at = company.get("changed_at")
    # last_vectorized = company.get("last_vectorized")

    # if changed_at and last_vectorized:
    #     try:
    #         if isinstance(changed_at, str):
    #             changed_at = datetime.datetime(changed_at)
    #         if isinstance(last_vectorized, str):
    #             last_vectorized = datetime.datetime(last_vectorized)

    #         if changed_at <= last_vectorized:
    #             print(f"Skipping {domain}: No changes since last vectorization.")
    #             continue
    #     except Exception as e:
    #         print(f"Date parse error for {domain}: {e}")

    vector_text = prepare_vector_text(company)
    if not vector_text:
        print(f"Skipping {domain}: No vector text.")
        continue

    try:
        embedding = model.encode(vector_text)
        if embedding is None or not isinstance(embedding, (list, np.ndarray)) or np.all(np.array(embedding) == 0):
            print(f"Skipping {domain}: Invalid embedding.")
            continue

        embedding = np.array(embedding, dtype=np.float32).tolist()
        point_id = str(uuid.uuid5(uuid.NAMESPACE_DNS, f"company:{domain}"))

        point = PointStruct(
            id=point_id,
            vector=embedding,
            payload={
                "company_id": str(company.get("_id")),
                "icp_id": str(company.get("icp_id", "")),
                "industry": normalize_list(company.get("industries")),
                "location": company.get("hq_location", {}).get("country", "").lower(),
                "tech_stack": normalize_list(company.get("tech_stack"))
            }
        )
        points.append(point)
    except Exception as e:
        print(f"Error vectorizing {domain}: {e}")

# === Upsert all vectors into Qdrant ===
if points:
    qdrant.upsert(collection_name="company_vectors", points=points)
    print(f"Upserted {len(points)} companies to Qdrant.")
else:
    print("No valid company vectors to upsert.")


/home/sagar/pipeline OS/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Upserted 5 companies to Qdrant.


In [14]:
# === Helper: Normalize and join values ===
def normalize_list(value):
    if isinstance(value, str):
        return [value.lower()]
    if isinstance(value, list):
        return list({v.strip().lower() for v in value if isinstance(v, str)})
    return []

# === Load active ICPs ===
latest_icp = list(icp_profiles.aggregate(pipeline))
icp_points = []

for icp in latest_icp:
    icp_id = str(icp["_id"])
    filters = icp.get("filters", {})
    text_parts = []

    # Normalize and extract fields
    # FIXED
    industry = normalize_list(filters.get("industry", []))
    locations = normalize_list(filters.get("locations", []))
    tech_stack = normalize_list(filters.get("tech_stack", []))


    text_parts += industry + locations + tech_stack
    vector_text = " ".join(text_parts).strip()

    if not vector_text:
        continue
    print(f"[{icp_id}] vector_text: '{vector_text}' | filters: {filters}")

    try:
        embedding = model.encode(vector_text)
        if embedding is None or not isinstance(embedding, (list, np.ndarray)) or np.all(np.array(embedding) == 0):
            continue

        embedding = np.array(embedding, dtype=np.float32).tolist()
        point_id = str(uuid.uuid5(uuid.NAMESPACE_DNS, f"icp:{icp_id}"))

        point = PointStruct(
            id=point_id,
            vector=embedding,
            payload={
                "icp_id": icp_id,
                "industry": industry,
                "location": locations,
                "tech_stack": tech_stack
            }
        )
        icp_points.append(point)

    except Exception as e:
        print(f"Error encoding ICP {icp_id}: {e}")

# === Upsert to Qdrant ===
if icp_points:
    qdrant.upsert(collection_name="icp_vectors", points=icp_points)
    print(f"Upserted {len(icp_points)} ICP profiles to Qdrant.")
else:
    print("No valid ICP vectors to upsert.")
    print(f"[{icp_id}] vector_text: '{vector_text}' | filters: {filters}")
    print(f"[{icp_id}] Embedding generated: {embedding is not None} | Zero vector: {np.all(np.array(embedding)==0) if embedding is not None else 'N/A'}")



[6899cb3aafa62213309e0357] vector_text: 'ai software usa canada python aws' | filters: {'locations': ['usa', 'canada'], 'tech_stack': ['AWS', 'python'], 'keywords': ['ai, ml'], 'designations': ['hr'], 'department': ['tect', 'aq', 'it'], 'industry': ['software', 'ai'], 'employee_count': {'min': 1, 'max': 500}, 'founded_after': 2019}
Upserted 1 ICP profiles to Qdrant.


#### cos_sim scoring:

In [15]:
# WEIGHTS = {
#     "industry": 20,
#     "employee_count": 15,
#     "location": 15,
#     "tech_stack": 25,
#     "keywords": 10,
#     "founded_year": 5,
#     "github_signal": 10
# }
 
# # --- 1. Get latest active ICP ---
# latest_icp = list(icp_profiles.aggregate(pipeline))
 
# if not latest_icp:
#     print("❌ No active ICP found.")
#     exit()
# icp = latest_icp[0]
# icp_id = str(icp["_id"])
# icp_version = icp.get("version", 1)
 
# # --- 2. Get ICP vector from Qdrant ---
 
# icp_vector_resp = qdrant.retrieve(
#     collection_name="icp_vectors",
#     ids= [str(uuid.uuid5(uuid.NAMESPACE_DNS, f"icp:{icp_id}"))],
#     with_vectors=True
# )
 
# if not icp_vector_resp or not icp_vector_resp[0].vector:
#     print("❌ ICP vector not found.")
#     exit()
 
# icp_embedding = icp_vector_resp[0].vector
 
# # --- 3. Get all company vectors from Qdrant ---
# search_result = qdrant.query_points(
#     collection_name="company_vectors",
#     query=icp_embedding,
#     limit=10000,
#     with_payload=True,
#     with_vectors=True
# )
# print(f"Total points returned from Qdrant: {len(search_result.points)}")
 
# icp_founded_after = icp.get("filters", {}).get("founded_after", 2015)
 
# # --- 4. Scoring Rule Function () ---
# def rule_score(company_doc, icp_tokens, icp_founded_after):
#     rule_score = 0
#     breakdown = {k: 0 for k in WEIGHTS}
#     def safe_list(field):
#         value = company_doc.get(field, [])
#         if isinstance(value, str): return [value]
#         if isinstance(value, list): return value
#         return []
    
#     # Match tokens in industry, location, tech stack, keywords
#     for field, key in [("industries", "industry"), ("hq_location", "location"),
#                        ("tech_stack", "tech_stack"), ("keywords", "keywords")]:
#         values = (
#             safe_list(field) if field != "hq_location"
#             else list(company_doc.get("hq_location", {}).values())
#         )
#         if any(token in str(v).lower() for v in values for token in icp_tokens):
#             rule_score += WEIGHTS[key]
#             breakdown[key] = WEIGHTS[key]
 
#     # Match employee count
#     emp = company_doc.get("employee_count_estimate", {})
#     if emp.get("min", 0) >= 10 and emp.get("max", 0) <= 500:
#         rule_score += WEIGHTS["employee_count"]
#         breakdown["employee_count"] = WEIGHTS["employee_count"]
 
#     # Founded year match
#     if isinstance(company_doc.get("founded_year"), int) and company_doc["founded_year"] >= icp_founded_after:
#         rule_score += WEIGHTS["founded_year"]
#         breakdown["founded_year"] = WEIGHTS["founded_year"]
 
#     # GitHub presence
#     urls = company_doc.get("source_urls", [])
#     if any("github" in url.lower() for url in urls):
#         rule_score += WEIGHTS["github_signal"]
#         breakdown["github_signal"] = WEIGHTS["github_signal"]
 
#     return rule_score, breakdown
 
# # --- 5. Tokenize ICP filters ---
# icp_tokens = [token.lower().strip() for token in f"{' '.join(icp.get('filters', {}).get('industries', []))} {' '.join(icp.get('filters', {}).get('locations', []))}".split()]
 
# # --- 6. Score companies using util.cos_sim() ---
# scored_results_util = []
# icp_tensor = torch.tensor(model.encode([' '.join(icp_tokens)]), dtype=torch.float32)  # Convert ICP to tensor
 
# for point in search_result.points:
#     payload = point.payload
#     company_id = payload.get("company_id")
#     company_vector = point.vector
    
#     # Calculate cosine similarity using util.cos_sim()
#     company_tensor = torch.tensor([company_vector], dtype=torch.float32)
#     similarity = util.cos_sim(icp_tensor, company_tensor)[0][0].item()
    
#     try:
#         if isinstance(company_id, str):
#             company_id = ObjectId(company_id)
#     except Exception as e:
#         print(f'{company_id} have {e}')
#         continue 
 
#     company = discovered_companies.find_one({"_id": company_id})
#     if not company:
#         continue
        
#     rule, breakdown = rule_score(company, icp_tokens, icp_founded_after)
#     if rule == 0:
#         final_score = round(similarity * 0.1, 4)
#     else:
#         final_score = round(similarity * rule, 4)
    
#     breakdown["vector_similarity"] = round(similarity, 4)
 
#     scored_doc = {
#         "company_id": company_id,
#         "icp_id": icp_id,
#         "icp_version": icp_version,
#         "final_score": final_score,
#         "breakdown": breakdown,
#         "weights": WEIGHTS,
#         "last_scored": datetime.now(timezone.utc),
#         "method": "util_cos_sim"
#     }
    
#     # Store in a different collection or add method flag
#     scored_companies.update_one(
#         {"company_id": company_id, "icp_id": icp_id, "method": "util_cos_sim"},
#         {"$set": scored_doc},
#         upsert=True
#     )
#     scored_results_util.append((company.get("domain", "unknown"), final_score))
 
# # --- 7. Print Top Results ---
# print("Scoring complete using util.cos_sim(). Top results:")
# for domain, score in sorted(scored_results_util, key=lambda x: x[1], reverse=True)[:20]:
#     print(f"{domain} → Score: {score}")
 

## dicovered_comanies

{
  "_id": {
    "$oid": "68890042ada3746b1bacf2a5"
  },
  "name": "Flowcast AI solutions",
  "domain": "flowcast.ai",
  "industries": [
    "Artificial Intelligence",
    "FinTech"
  ],
  "employee_count_estimate": {
    "min": 50,
    "max": 200
  },
  "hq_location": {
    "city": "San Francisco",
    "country": "USA"
  },
  "founded_year": 2019,
  "tech_stack": [
    "Python",
    "AWS",
    "LangChain"
  ],
  "social_metrics": {
    "followers": {
      "linkedin": 1500,
      "twitter": 600
    }
  },
  "source_urls": [
    "https://linkedin.com/company/flowcast-ai"
  ],
  "last_scraped": {
    "$date": "2025-07-27T11:00:00.000Z"
  },
  "last_scored": null,
  "last_vectorized": {
    "$date": "2025-08-04T10:32:19.574Z"
  },
  "changed_at": {
    "$date": "2025-08-03T10:00:00.000Z"
  },
  "icp_id": {
    "$oid": "64c4a23b69a84f2b0b789cde"
  },
  "icp_version": 1
}

-------------------------------------

{
  "_id": {
    "$oid": "68890042ada3746b1bacf2a6"
  },
  "name": "SynthAI Inc.",
  "domain": "ssynthai.io",
  "industries": [
    "Artificial Intelligence",
    "SaaS"
  ],
  "employee_count_estimate": {
    "min": 20,
    "max": 80
  },
  "hq_location": {
    "city": "Toronto",
    "country": "Canada"
  },
  "founded_year": 2021,
  "tech_stack": [
    "Python",
    "GCP",
    "HuggingFace"
  ],
  "social_metrics": {
    "followers": {
      "linkedin": 700,
      "twitter": 1200
    }
  },
  "source_urls": [
    "https://linkedin.com/company/synthai"
  ],
  "last_scraped": {
    "$date": "2025-07-28T08:30:00.000Z"
  },
  "last_scored": {
    "$date": "2025-07-27T12:00:00.000Z"
  },
  "last_vectorized": {
    "$date": "2025-08-04T10:32:19.593Z"
  },
  "changed_at": {
    "$date": "2025-07-27T13:00:00.000Z"
  },
  "icp_id": {
    "$oid": "64c4a23b69a84f2b0b789cdf"
  },
  "icp_version": 1
}

-------------------------------------

{
  "_id": {
    "$oid": "68890042ada3746b1bacf2a7"
  },
  "name": "DocStream AI",
  "domain": "ddocstream.com",
  "industries": [
    "SaaS",
    "Document Management"
  ],
  "employee_count_estimate": {
    "min": 100,
    "max": 250
  },
  "hq_location": {
    "city": "Austin",
    "country": "USA"
  },
  "founded_year": 2016,
  "tech_stack": [
    "Node.js",
    "MongoDB",
    "AWS"
  ],
  "social_metrics": {
    "followers": {
      "linkedin": 300,
      "twitter": 90
    }
  },
  "source_urls": [
    "https://linkedin.com/company/docstream"
  ],
  "last_scraped": {
    "$date": "2025-07-25T09:00:00.000Z"
  },
  "last_scored": {
    "$date": "2025-07-26T09:30:00.000Z"
  },
  "last_vectorized": {
    "$date": "2025-08-04T10:32:19.613Z"
  },
  "changed_at": {
    "$date": "2025-07-25T14:00:00.000Z"
  },
  "icp_id": {
    "$oid": "64c4a23b69a84f2b0b789ce0"
  },
  "icp_version": 1
}

-----------------------

{
  "_id": {
    "$oid": "68890042ada3746b1bacf2a8"
  },
  "name": "NeoGen Cloud",
  "domain": "neogencloud.com",
  "industries": [
    "Cloud",
    "AI"
  ],
  "employee_count_estimate": {
    "min": 5,
    "max": 30
  },
  "hq_location": {
    "city": "Vancouver",
    "country": "Canada"
  },
  "founded_year": 2022,
  "tech_stack": [
    "Azure",
    "Docker",
    "LangChain"
  ],
  "social_metrics": {
    "followers": {
      "linkedin": 250,
      "twitter": 50
    }
  },
  "source_urls": [
    "https://linkedin.com/company/neogencloud"
  ],
  "last_scraped": {
    "$date": "2025-07-29T09:00:00.000Z"
  },
  "last_scored": null,
  "last_vectorized": {
    "$date": "2025-08-04T10:32:19.630Z"
  },
  "changed_at": {
    "$date": "2025-07-29T08:50:00.000Z"
  },
  "icp_id": {
    "$oid": "64c4a23b69a84f2b0b789ce1"
  },
  "icp_version": 2
}


--------------------------

{
  "_id": {
    "$oid": "68890042ada3746b1bacf2a9"
  },
  "name": "Retico AI",
  "domain": "retico.ai",
  "industries": [
    "Artificial Intelligence"
  ],
  "employee_count_estimate": {
    "min": 500,
    "max": 1000
  },
  "hq_location": {
    "city": "New York",
    "country": "USA"
  },
  "founded_year": 2015,
  "tech_stack": [
    "Python",
    "AWS",
    "Docker"
  ],
  "social_metrics": {
    "followers": {
      "linkedin": 5000,
      "twitter": 3000
    }
  },
  "source_urls": [
    "https://linkedin.com/company/retico"
  ],
  "last_scraped": {
    "$date": "2025-07-28T12:00:00.000Z"
  },
  "last_scored": {
    "$date": "2025-07-27T12:00:00.000Z"
  },
  "last_vectorized": {
    "$date": "2025-08-04T10:32:19.644Z"
  },
  "changed_at": {
    "$date": "2025-07-28T12:00:00.000Z"
  },
  "icp_id": {
    "$oid": "64c4a23b69a84f2b0b789ce2"
  },
  "icp_version": 3
}

## Test Code

In [23]:
WEIGHTS = {
    "industry": 20,
    "employee_count": 15,
    "location": 15,
    "tech_stack": 25,
    "keywords": 10,
    "founded_year": 5,
    "github_signal": 10
}
 
# --- 1. Get latest active ICP ---
latest_icp = list(icp_profiles.aggregate(pipeline))
 
if not latest_icp:
    print("❌ No active ICP found.")
    exit()
icp = latest_icp[0]
icp_id = str(icp["_id"])
icp_version = icp.get("version", 1)
 
# --- 2. Get ICP vector from Qdrant ---
 
icp_vector_resp = qdrant.retrieve(
    collection_name="icp_vectors",
    ids= [str(uuid.uuid5(uuid.NAMESPACE_DNS, f"icp:{icp_id}"))],
    with_vectors=True
)
 
if not icp_vector_resp or not icp_vector_resp[0].vector:
    print("❌ ICP vector not found.")
    exit()
 
icp_embedding = icp_vector_resp[0].vector
 
# --- 3. Get all company vectors from Qdrant ---
search_result = qdrant.query_points(
    collection_name="company_vectors",
    query=icp_embedding,
    limit=10000,
    with_payload=True,
    with_vectors=True
)
print(f"Total points returned from Qdrant: {len(search_result.points)}")
 
# icp_founded_after = icp.get("filters", {}).get("founded_after", 2015)
 
# --- 4. Scoring Rule Function () ---
def rule_score(company_doc, icp_tokens, icp_field_tokens):
    rule_score = 0
    breakdown = {k: 0 for k in WEIGHTS}
    def safe_list(field):
        value = company_doc.get(field, [])
        if isinstance(value, str): return [value]
        if isinstance(value, list): return value
        return []
    
    # Match tokens in industry, location, tech stack, keywords
    for field, key in [("industries", "industry"), ("hq_location", "location"),
                       ("tech_stack", "tech_stack"), ("keywords", "keywords")]:
        values = (
            safe_list(field) if field != "hq_location"
            else list(company_doc.get("hq_location", {}).values())
        )
        if any(token in str(v).lower() for v in values for token in icp_tokens):
            rule_score += WEIGHTS[key]
            breakdown[key] = WEIGHTS[key]
 
    # Match employee count
    emp = company_doc.get("employee_count_estimate", {})
    if emp.get("min", 0) >= 10 and emp.get("max", 0) <= 500:
        rule_score += WEIGHTS["employee_count"]
        breakdown["employee_count"] = WEIGHTS["employee_count"]
 
    # Founded year match
    if isinstance(company_doc.get("founded_year"), int) and company_doc["founded_year"] >= icp_field_tokens.get("founded_after") :
        rule_score += WEIGHTS["founded_year"]
        breakdown["founded_year"] = WEIGHTS["founded_year"]
 
    # GitHub presence
    urls = company_doc.get("source_urls", [])
    if any("github" in url.lower() for url in urls):
        rule_score += WEIGHTS["github_signal"]
        breakdown["github_signal"] = WEIGHTS["github_signal"]
 
    return rule_score, breakdown
 
# --- 5. Tokenize ICP filters ---
icp_filters = icp.get('filters', {})
icp_tokens =[]
icp_field_tokens = {}

# extract industry tokens
for field_name in ['industry', 'industries']:
    if field_name in icp_filters:
        industries = icp_filters[field_name]
        if isinstance(industries, list):
            for industry in industries:
                tokens = str(industry).lower().split()
                icp_tokens.extend(tokens)
        elif isinstance(industries, str):
            tokens = industries.lower().split()
            icp_tokens.extend(tokens)

# Extract tech stack tokens
if 'tech_stack' in icp_filters:
    tech_stack = icp_filters['tech_stack']
    if isinstance(tech_stack, list):
        for tech in tech_stack:
            # Keep full tech names for exact matching
            icp_tokens.append(str(tech).lower())
    elif isinstance(tech_stack, str):
        icp_tokens.append(tech_stack.lower())
 
# Extract keyword tokens
if 'keywords' in icp_filters:
    keywords = icp_filters['keywords']
    if isinstance(keywords, list):
        for keyword in keywords:
            icp_tokens.append(str(keyword).lower())
    elif isinstance(keywords, str):
        icp_tokens.append(keywords.lower())


# Extract location tokens
for field_name in ['location', 'locations']:
    if field_name in icp_filters:
        locations = icp_filters[field_name]
        if isinstance(locations, list):
            for location in locations:
                tokens = str(location).lower().split()
                icp_tokens.extend(tokens)
        elif isinstance(locations, str):
            tokens = locations.lower().split()
            icp_tokens.extend(tokens)



# Employee count
if 'employee_count' in icp_filters:
    emp_filter = icp_filters['employee_count']
    if isinstance(emp_filter, dict):
        icp_field_tokens['employee_count'] = emp_filter  # keep structured for later numeric match
        if 'min' in emp_filter:
            icp_tokens.append(f"emp_min_{emp_filter['min']}")
        if 'max' in emp_filter:
            icp_tokens.append(f"emp_max_{emp_filter['max']}")

# Founded after
if 'founded_after' in icp_filters:
    founded_year = icp_filters['founded_after']
    icp_field_tokens['founded_after'] = founded_year  # keep structured
    icp_tokens.append(f"founded_after_{founded_year}")




# Clean up tokens
icp_tokens = list(set(token.strip() for token in icp_tokens if token.strip()))

print(f"✅ Extracted ICP tokens: {icp_tokens}")
print(f"✅ Field-specific tokens: {icp_field_tokens}")



# --- 6. Score companies using util.cos_sim() ---
scored_results_util = []
icp_tensor = torch.tensor([icp_embedding], dtype=torch.float32)  # Convert ICP to tensor

for point in search_result.points:
    payload = point.payload
    company_id = payload.get("company_id")
    company_vector = point.vector

    # Calculate cosine similarity using util.cos_sim()
    company_tensor = torch.tensor([company_vector], dtype=torch.float32)
    raw_similarity = util.cos_sim(icp_tensor, company_tensor)[0][0].item()
    vector_similarity_score = ((raw_similarity + 1) / 2) * 100  # Normalize to 0–100

    try:
        if isinstance(company_id, str):
            company_id = ObjectId(company_id)
    except Exception as e:
        print(f'{company_id} has invalid ObjectId: {e}')
        continue 

    company = discovered_companies.find_one({"_id": company_id})
    if not company:
        continue

    # Apply rule-based scoring
    rule_score_total, breakdown = rule_score(company, icp_tokens, icp_field_tokens)
    print("\n",breakdown)
    print(rule_score)
    # Final score weights
    vector_weight = 0.4
    rule_weight = 0.6

    final_score = round(
        (vector_similarity_score * vector_weight) + (rule_score_total * rule_weight),
        2
    )

    breakdown["vector_similarity"] = round(vector_similarity_score, 2)

    scored_doc = {
        "company_id": company_id,
        "icp_id": icp_id,
        "icp_version": icp_version,
        "final_score": final_score,
        "breakdown": breakdown,
        "weights": WEIGHTS,
        "last_scored": datetime.now(timezone.utc),
        "method": "util_cos_sim"
    }

    scored_companies.update_one(
        {"company_id": company_id, "icp_id": icp_id, "method": "util_cos_sim"},
        {"$set": scored_doc},
        upsert=True
    )

    scored_results_util.append((company.get("domain", "unknown"), final_score))

# --- 7. Print Top Results ---
print(f"📈 Total companies scored: {len(scored_results_util)}")

sorted_results = sorted(scored_results_util, key=lambda x: x[1], reverse=True)

print("\n🏆 TOP RESULTS (Weighted Average Method):")
for i, (domain, score) in enumerate(sorted_results[:10], 1):
    print(f"{i:2d}. {domain:20s} → {score:6.2f}%")


Total points returned from Qdrant: 5
✅ Extracted ICP tokens: ['emp_min_1', 'founded_after_2019', 'usa', 'ai', 'software', 'ai, ml', 'canada', 'python', 'emp_max_500', 'aws']
✅ Field-specific tokens: {'employee_count': {'min': 1, 'max': 500}, 'founded_after': 2019}

 {'industry': 0, 'employee_count': 15, 'location': 15, 'tech_stack': 25, 'keywords': 0, 'founded_year': 5, 'github_signal': 0}
<function rule_score at 0x7717ca1b7640>

 {'industry': 0, 'employee_count': 0, 'location': 15, 'tech_stack': 25, 'keywords': 0, 'founded_year': 0, 'github_signal': 0}
<function rule_score at 0x7717ca1b7640>

 {'industry': 0, 'employee_count': 15, 'location': 15, 'tech_stack': 25, 'keywords': 0, 'founded_year': 5, 'github_signal': 0}
<function rule_score at 0x7717ca1b7640>

 {'industry': 20, 'employee_count': 0, 'location': 15, 'tech_stack': 25, 'keywords': 0, 'founded_year': 5, 'github_signal': 0}
<function rule_score at 0x7717ca1b7640>

 {'industry': 0, 'employee_count': 15, 'location': 15, 'tech_st

In [17]:
icp_filters

{'locations': ['usa', 'canada'],
 'tech_stack': ['AWS', 'python'],
 'keywords': ['ai, ml'],
 'designations': ['hr'],
 'department': ['tect', 'aq', 'it'],
 'industry': ['software', 'ai'],
 'employee_count': {'min': 1, 'max': 500},
 'founded_after': 2019}

## test code 2

In [18]:
WEIGHTS = {
    "industry": 20,
    "employee_count": 15,
    "location": 15,
    "tech_stack": 25,
    "keywords": 10,
    "founded_after": 5,
    "github_signal": 10
}
 
# --- 1. Get latest active ICP ---
latest_icp = list(icp_profiles.aggregate(pipeline))
 
if not latest_icp:
    print("❌ No active ICP found.")
    exit()
icp = latest_icp[0]
icp_id = str(icp["_id"])
icp_version = icp.get("version", 1)
 
# --- 2. Get ICP vector from Qdrant ---
 
icp_vector_resp = qdrant.retrieve(
    collection_name="icp_vectors",
    ids= [str(uuid.uuid5(uuid.NAMESPACE_DNS, f"icp:{icp_id}"))],
    with_vectors=True
)
 
if not icp_vector_resp or not icp_vector_resp[0].vector:
    print("❌ ICP vector not found.")
    exit()
 
icp_embedding = icp_vector_resp[0].vector
 
# --- 3. Get all company vectors from Qdrant ---
search_result = qdrant.query_points(
    collection_name="company_vectors",
    query=icp_embedding,
    limit=10000,
    with_payload=True,
    with_vectors=True
)
print(f"Total points returned from Qdrant: {len(search_result.points)}")
 
 



# --- 5. Tokenize ICP filters ---
icp_filters = icp.get('filters', {})
icp_tokens = []
icp_field_tokens = {}  # Track tokens by field for better debugging
 
# Extract industry tokens
for field_name in ['industry', 'industries']:
    if field_name in icp_filters:
        industries = icp_filters[field_name]
        field_tokens = []
        if isinstance(industries, list):
            for industry in industries:
                tokens = str(industry).lower().split()
                field_tokens.extend(tokens)
                icp_tokens.extend(tokens)
        elif isinstance(industries, str):
            tokens = industries.lower().split()
            field_tokens.extend(tokens)
            icp_tokens.extend(tokens)
        icp_field_tokens['industry'] = field_tokens
 
# Extract location tokens
for field_name in ['location', 'locations']:
    if field_name in icp_filters:
        locations = icp_filters[field_name]
        field_tokens = []
        if isinstance(locations, list):
            for location in locations:
                tokens = str(location).lower().split()
                field_tokens.extend(tokens)
                icp_tokens.extend(tokens)
        elif isinstance(locations, str):
            tokens = locations.lower().split()
            field_tokens.extend(tokens)
            icp_tokens.extend(tokens)
        icp_field_tokens['location'] = field_tokens
 
# Extract tech stack tokens
if 'tech_stack' in icp_filters:
    tech_stack = icp_filters['tech_stack']
    field_tokens = []
    if isinstance(tech_stack, list):
        for tech in tech_stack:
            # Keep full tech names for exact matching
            token = str(tech).lower()
            field_tokens.append(token)
            icp_tokens.append(token)
    elif isinstance(tech_stack, str):
        token = tech_stack.lower()
        field_tokens.append(token)
        icp_tokens.append(token)
    icp_field_tokens['tech_stack'] = field_tokens
 
# Extract keyword tokens
if 'keywords' in icp_filters:
    keywords = icp_filters['keywords']
    field_tokens = []
    if isinstance(keywords, list):
        for keyword in keywords:
            token = str(keyword).lower()
            field_tokens.append(token)
            icp_tokens.append(token)
    elif isinstance(keywords, str):
        token = keywords.lower()
        field_tokens.append(token)
        icp_tokens.append(token)
    icp_field_tokens['keywords'] = field_tokens
 
# Extract employee count range (for matching logic)
if 'employee_count' in icp_filters:
    emp_filter = icp_filters['employee_count']
    icp_field_tokens['employee_count'] = emp_filter  # Store the range dict
 
# Extract founded year (for matching logic)
# if 'founded_after' in icp:
#     icp_field_tokens['founded_after'] = icp['founded_after']
if 'founded_after' in icp_filters:
    icp_field_tokens['founded_after'] = icp_filters['founded_after']
 
# Clean up tokens
icp_tokens = [token.strip() for token in icp_tokens if token.strip()]
icp_tokens = list(set(icp_tokens))  # Remove duplicates
 
print(f"✅ Extracted ICP tokens: {icp_tokens}")
print(f"✅ Field-specific tokens: {icp_field_tokens}")
 
 

# icp_tokens = list(set([token for token in icp_tokens if token.strip()]))


# icp_founded_after = icp.get("filters", {}).get("founded_after", 2015)

# --- 4. Scoring Rule Function () ---
def rule_score(company_doc, icp_tokens, icp_field_tokens):
    rule_score = 0
    breakdown = {k: 0 for k in WEIGHTS}
    def safe_list(field):
        value = company_doc.get(field, [])
        if isinstance(value, str): return [value]
        if isinstance(value, list): return value
        return []
    
    # 1. INDUSTRY MATCHING
    if icp_field_tokens and 'industry' in icp_field_tokens:
        industry_tokens = icp_field_tokens['industry']
        company_industries = safe_list("industries")
        if any(token in str(v).lower() for v in company_industries for token in industry_tokens):
            rule_score += WEIGHTS["industry"]
            breakdown["industry"] = WEIGHTS["industry"]
    
    # 2. LOCATION MATCHING  
    if icp_field_tokens and 'location' in icp_field_tokens:
        location_tokens = icp_field_tokens['location']
        company_location_values = list(company_doc.get("hq_location", {}).values())
        if any(token in str(v).lower() for v in company_location_values for token in location_tokens):
            rule_score += WEIGHTS["location"]
            breakdown["location"] = WEIGHTS["location"]
    
    # 3. TECH STACK MATCHING
    if icp_field_tokens and 'tech_stack' in icp_field_tokens:
        tech_tokens = icp_field_tokens['tech_stack']
        company_tech = safe_list("tech_stack")
        if any(token in str(v).lower() for v in company_tech for token in tech_tokens):
            rule_score += WEIGHTS["tech_stack"]
            breakdown["tech_stack"] = WEIGHTS["tech_stack"]
    
    # 4. KEYWORDS MATCHING
    if icp_field_tokens and 'keywords' in icp_field_tokens:
        keyword_tokens = icp_field_tokens['keywords']
        text_fields = [
            company_doc.get("description", ""),
            company_doc.get("name", ""),
            " ".join(safe_list("industries")),
            " ".join(safe_list("tech_stack"))
        ]
        company_text = " ".join(text_fields).lower()
        if any(token in company_text for token in keyword_tokens):
            rule_score += WEIGHTS["keywords"]
            breakdown["keywords"] = WEIGHTS["keywords"]
    # 5. EMPLOYEE COUNT MATCHING (DYNAMIC)
    emp = company_doc.get("employee_count_estimate", {})
    emp_min = emp.get("min", 0)
    emp_max = emp.get("max", float('inf'))
    
    if icp_field_tokens and 'employee_count' in icp_field_tokens:
        icp_emp = icp_field_tokens['employee_count']
        if isinstance(icp_emp, dict):
            icp_min = icp_emp.get('min')
            icp_max = icp_emp.get('max')
            
            # Check if company employee count falls within the ICP range
            if not (emp_max< icp_min or emp_min > icp_max):
                rule_score += WEIGHTS["employee_count"]
                breakdown["employee_count"] = WEIGHTS["employee_count"]
    # 6. FOUNDED YEAR MATCHING (DYNAMIC)
    company_founded_year = company_doc.get("founded_year")  # FIXED field name

    if icp_field_tokens and 'founded_after' in icp_field_tokens:
        year_threshold = icp_field_tokens['founded_after']

    if isinstance(company_founded_year, int) and company_founded_year >= year_threshold:
        rule_score += WEIGHTS["founded_after"]
        breakdown["founded_after"] = WEIGHTS["founded_after"]



    # 7. GITHUB SIGNAL MATCHING
    urls = company_doc.get("source_urls", [])
    if any("github" in url.lower() for url in urls):
        rule_score += WEIGHTS["github_signal"]
        breakdown["github_signal"] = WEIGHTS["github_signal"]
    return rule_score, breakdown
 


# --- 6. Score companies using util.cos_sim() ---
scored_results_util = []
icp_tensor = torch.tensor([icp_embedding], dtype=torch.float32)  # Convert ICP to tensor

for point in search_result.points:
    payload = point.payload
    company_id = payload.get("company_id")
    company_vector = point.vector

    # Calculate cosine similarity using util.cos_sim()
    company_tensor = torch.tensor([company_vector], dtype=torch.float32)
    raw_similarity = util.cos_sim(icp_tensor, company_tensor)[0][0].item()
    vector_similarity_score = ((raw_similarity + 1) / 2) * 100  # Normalize to 0–100

    try:
        if isinstance(company_id, str):
            company_id = ObjectId(company_id)
    except Exception as e:
        print(f'{company_id} has invalid ObjectId: {e}')
        continue 


# Fetch all companies from the collection
for company in discovered_companies.find({}):
    # Apply rule-based scoring
    score, breakdown = rule_score(company, icp_tokens, icp_field_tokens)



    # Final score weights
    vector_weight = 0.4
    rule_weight = 0.6

    final_score = round(
        (vector_similarity_score * vector_weight) + (score * rule_weight),
        2
    )

    breakdown["vector_similarity"] = round(vector_similarity_score, 2)

    scored_doc = {
        "company_id": company_id,
        "icp_id": icp_id,
        "icp_version": icp_version,
        "final_score": final_score,
        "breakdown": breakdown,
        "weights": WEIGHTS,
        "last_scored": datetime.now(timezone.utc),
        "method": "util_cos_sim"
    }

    scored_companies.update_one(
        {"company_id": company_id, "icp_id": icp_id, "method": "util_cos_sim"},
        {"$set": scored_doc},
        upsert=True
    )

    scored_results_util.append((company.get("domain", "unknown"), final_score))

# --- 7. Print Top Results ---
print(f"📈 Total companies scored: {len(scored_results_util)}")

sorted_results = sorted(scored_results_util, key=lambda x: x[1], reverse=True)

print("\n🏆 TOP RESULTS (Weighted Average Method):")
for i, (domain, score) in enumerate(sorted_results[:10], 1):
    print(f"{i:2d}. {domain:20s} → {score:6.2f}%")


Total points returned from Qdrant: 5
✅ Extracted ICP tokens: ['usa', 'ai', 'software', 'ai, ml', 'canada', 'python', 'aws']
✅ Field-specific tokens: {'industry': ['software', 'ai'], 'location': ['usa', 'canada'], 'tech_stack': ['aws', 'python'], 'keywords': ['ai, ml'], 'employee_count': {'min': 1, 'max': 500}, 'founded_after': 2019}
📈 Total companies scored: 5

🏆 TOP RESULTS (Weighted Average Method):
 1. flowcast.ai          →  62.94%
 2. ssynthai.io          →  62.94%
 3. ddocstream.com       →  59.94%
 4. neogencloud.com      →  59.94%
 5. retico.ai            →  59.94%


In [ ]:
for company in discovered_companies.find({}):
    score, breakdown = rule_score(company_doc=company, icp_tokens=icp_tokens, icp_field_tokens=icp_field_tokens)
    print(breakdown)

{'industry': 0, 'employee_count': 15, 'location': 0, 'tech_stack': 25, 'keywords': 0, 'founded_after': 5, 'github_signal': 0}
{'industry': 0, 'employee_count': 15, 'location': 0, 'tech_stack': 25, 'keywords': 0, 'founded_after': 5, 'github_signal': 0}
{'industry': 0, 'employee_count': 15, 'location': 0, 'tech_stack': 0, 'keywords': 0, 'founded_after': 0, 'github_signal': 0}
{'industry': 0, 'employee_count': 15, 'location': 0, 'tech_stack': 0, 'keywords': 0, 'founded_after': 5, 'github_signal': 0}
{'industry': 0, 'employee_count': 15, 'location': 0, 'tech_stack': 25, 'keywords': 0, 'founded_after': 0, 'github_signal': 0}


#### Store to MongoDB :

In [19]:


scored_companies = db['scored_companies']

# Call this after scoring each company
def store_scored_company(company, score_tuple, icp_id, icp_version=1):
    score, breakdown, similarity = score_tuple

    company_oid = ObjectId(company["_id"])
    icp_oid = ObjectId(icp_id)

    doc = {
        "company_id": company_oid,
        "icp_id": icp_oid,
        "score": float(score),
        "breakdown": {
            **breakdown,
            "vector_similarity": float(similarity)
        },
        "last_scored": datetime.now(timezone.utc),
        "icp_version": icp_version
    }

    # 🛠 Upsert safely by matching ObjectId fields
    scored_companies.update_one(
        {"company_id": company_oid, "icp_id": icp_oid},
        {"$set": doc},
        upsert=True
    )

# Store in a different collection or add method flag
    scored_companies.update_one(
        {"company_id": company_id, "icp_id": icp_id, "method": "util_cos_sim"},
        {"$set": scored_doc},
        upsert=True
    )

In [20]:
final_scores = []
similarities = []

for doc in scored_companies.find():
    similarities.append(doc["breakdown"]["vector_similarity"])
    final_scores.append(doc["final_score"])

corr, _ = pearsonr(similarities, final_scores)
print(f"Pearson correlation (similarity vs. final score): {round(corr, 3)}")



Pearson correlation (similarity vs. final score): 0.933


In [21]:
final_scores = []
similarities = []

for doc in scored_companies.find():
    similarities.append(doc["breakdown"]["vector_similarity"])
    final_scores.append(doc["final_score"])
    
corr, _ = spearmanr(similarities, final_scores)
print(f"Spearman correlation (similarity vs. final score): {round(corr, 3)}")

Spearman correlation (similarity vs. final score): 0.926
